# Implicits

* Implicit conversion
* Implicit parameters

## Implicit conversions

* Implicit conversion function is a function with exactly a single parameter that is also declared with the keyword `implicit`.

* Implicit conversions can be used to make existing classes richer(with even more functionality)

**NOTE**: To avoid compiler warning when using implicit conversions,
import `import scala.language.implicitConversions` or the compiler option
`-language:implicitConversions`


In [2]:
import scala.language.implicitConversions

import scala.language.implicitConversions

In [1]:
case class Temperature(celcius: Double) {
    def toFarenheit: Double = celcius * (9.0/5) + 32.0
}

implicit def temperatureConverter(celcius: Double): Temperature =
    Temperature(celcius)

// toFarenheit is a member of Temperature.
// compiler will look for types with this method
// and an implicit conversion function that is in scope
// to convert double to Temperature
println((0.0).toFarenheit)

32.0


defined class Temperature
defined function temperatureConverter

Uses of implicit conversion
* To enrich existing type, we first implement a type that accepts the
type to be enriched as its constructor argument and provides those rich
methods. Then we need to add an implicit conversion function to convert
the existing type to the rich type.

In [7]:
// Approach-1
// provide conversion function from existing type to rich type
class MyRichInt(val value: Int) {
    def upto(end: Int): Seq[Int] = {
        Stream.iterate(value)(v => v + 1).take(end).toVector
    }
    
    def till(end: Int): Seq[Int] = {
        Stream.iterate(value)(v => v + 1).take(end-1).toVector
    }
}

implicit def enrichInt(value: Int): MyRichInt = new MyRichInt(value)

println(1 upto 10)
println(1 till 10)

Vector(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)
Vector(1, 2, 3, 4, 5, 6, 7, 8, 9)


defined class MyRichInt
defined function enrichInt

In [8]:
// Approach-2
// Make the new rich type itself implicit
// Implicit class should have a primary constructor with exactly 
// a single parameter.
// But in jupyter notebooks, We cannot use AnyVal since almond scala kernel
// wraps this cell code in some internal class.
// Here I renamed to MyRichInt2, otherwise I would get
// ambiguous error as we would have both implicit class
// and the implicit function from the above cell in the same
// scope to do the implicit conversion.
implicit class MyRichInt2(val value: Int) {
    def upto2(end: Int): Seq[Int] = {
        Stream.iterate(value)(v => v + 1).take(end).toVector
    }
    
    def till2(end: Int): Seq[Int] = {
        Stream.iterate(value)(v => v + 1).take(end-1).toVector
    }
}

println(1 upto2 10)
println(1 till2 10)

Vector(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)
Vector(1, 2, 3, 4, 5, 6, 7, 8, 9)


defined class MyRichInt2

NOTE
* Its recommended that implicit classes extend `AnyVal` because it eliminates instance creation overhead. [Value classes scala docs](https://docs.scala-lang.org/overviews/core/value-classes.html#summary_of_limitations)
* Implicit classes cannot be top level classes. But both implicit and value classes can be placed inside an `object`.

Rules for importing implicits

* Look for implicit functions/classes inside the companion object of the source or the target type
* Implicit functions or classes in scope.
* For implicit conversions to be in scope, we should import them without the prefixes.
Implicits should be imported only at required places(localized importing)


Rules for implicit conversions
* If type of an expression differs from expected type
* If an object accesses a nonexistent member(ex: `1 to 10`)
* If an object invokes a method whose parameters don't match the given arguments

When implicit conversions **are not attempted**
* If code compiles without implicit conversions
* Multiple levels of implicit conversions are not performed.
* Ambiguous conversions cause error.

NOTE: Dump the implicit conversion used by the compiler using `scalac -Xprint:typer MyProg.scala`

## Implicit parameters

* Function and method arguments can be declared as implicits.
In such cases, the compiler looks for val/def defined in the current scope
that are marked as implicit and automatically passes those values to the function/method call.

* There can be only one implicit value for a given data type.

Compiler looks for val/def in the following places:
* First a val/def in the current scope without a prefix otherwise,
* In the companion object of a type that is associated with the desired type.

In [10]:
object SomeObject {
    implicit val greet: String = "Hello"
}

import SomeObject.greet

// Here the greetWord is picked up from vals in the scope
def greetPerson(name: String)(implicit greetWord: String) = {
    println(s"$greetWord, $name")
}

greetPerson("John")

Hello, John


defined object SomeObject
import SomeObject.greet

// Here the greetWord is picked up from vals in the scope

defined function greetPerson

In [14]:
case class GreetWord(word: String)

object GreetWord {
    // this implicit type should match the type of the implicit parameter.
    implicit val defaultGreetWord = GreetWord("Hi")
}

// If this variable is in scope, it would be picked up
// since it is commented, compiler will look for implicit inside
// the companion object of GreetWord
// implicit val anotherGreetInScope = GreetWord("Hey")

// Here the greetWord is picked up from vals in the scope
def greetPerson2(name: String)(implicit greetWord: GreetWord) = {
    println(s"${greetWord.word}, $name")
}

greetPerson2("John")

Hi, John


defined class GreetWord
defined object GreetWord
defined function greetPerson2

**NOTE**: Implicit parameters can be a lambda function as well (`def smaller[T](a: T, b: T)(implicit order: T => Ordered[T])`) which helps in type conversion, thereby acting as an implicit conversion that is in the scope of the function/method body. Refer 21.6 for example.